In [1]:
import cv2
import numpy as np
import datetime
import dlib #얼굴탐색
import pygame

from keras.models import load_model
from imutils import face_utils

pygame 2.0.0 (SDL 2.0.12, python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#얼굴 찾기 호출
#dlib.get_frontal_face_detector()로 생성된 
#기본 face_detecor 객체는 dlib.fhog_object_detector 객체다.
detector = dlib.get_frontal_face_detector()

#다운 받은 shape_predictor_68_face_landmarks.dat 불러오기
predictor = dlib.shape_predictor('./face_data/shape_predictor_68_face_landmarks.dat') #얼굴특징 뽑아내기 눈코입

#다운 받은 dlib_face_recognition_resnet_model_v1.dat 불러오기
facerec = dlib.face_recognition_model_v1('./face_data/dlib_face_recognition_resnet_model_v1.dat')

In [3]:
model = load_model('datas/models/left_eye.h5')

In [4]:
#VideoCapture(0) : 자신의 기기 웹캠 불러옴
stream = cv2.VideoCapture(0)

In [5]:
#sound
#import한 pygame 모듈 초기화시킴
pygame.init()
alarm = pygame.mixer.music
alarm.load('./datas/good_morning.ogg') # alarm에 음악 재생 파일 로드
# alarm.set_volume(0.7) #100프로는 1.0

In [6]:
def crop_eye(img, eye_points):
    x1, y1 = np.amin(eye_points, axis=0)
    x2, y2 = np.amax(eye_points, axis=0)
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2 #중앙

    w = (x2 - x1) * 1.2
    h = w * 26 / 34

    margin_x, margin_y = w / 2, h / 2

    min_x, min_y = int(cx - margin_x), int(cy - margin_y)
    max_x, max_y = int(cx + margin_x), int(cy + margin_y)

    eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(np.int)

    eye_img = img[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]

    return eye_img, eye_rect

In [ ]:
while(stream.isOpened()):
    ret, frame = stream.read()
    
    if ret:
        alarm_flags = [] #눈을 두명 모두 감으면 True, 두명 모두 눈을 뜨면 False, 1명 이상 true면 알람 실행
        now = datetime.datetime.now()
        
        #dets = detector(frame,1)
        
        gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

        faces = detector(gray)
        #print(dets)

#gray로 바꾸는 장점: 단일색인 데이터로 학습을 시키기도 했고, 데이터를 학습해 테스트하면 확률을 높일 수 있음
        for face in faces:
            shapes = predictor(gray, face)
            shapes = face_utils.shape_to_np(shapes) #얼굴 특징을 행렬연산하도록 만들어줌

            eye_img_l, eye_rect_l = crop_eye(gray, eye_points=shapes[36:42]) #왼쪽 눈 위치정보
            eye_img_r, eye_rect_r = crop_eye(gray, eye_points=shapes[42:48]) #오른쪽 눈 위치정보

            eye_img_l = cv2.resize(eye_img_l, dsize=(34,26))
            eye_img_r = cv2.resize(eye_img_r, dsize=(34,26))
            eye_img_r = cv2.flip(eye_img_r, flipCode=1)

            eye_input_l = eye_img_l.copy().reshape((1, 26, 34, 1)).astype(np.float32) / 255.
            eye_input_r = eye_img_r.copy().reshape((1, 26, 34, 1)).astype(np.float32) / 255.

            pred_l = model.predict(eye_input_l)
            pred_r = model.predict(eye_input_r)

           # print(pred_l, pred_r)

            state_l = 1 if pred_l > 0.1 else 0
            state_r = 1 if pred_r > 0.1 else 0
            
            #left
            if state_l == 1:
                cv2.rectangle(frame, pt1=tuple(eye_rect_l[0:2]), pt2=tuple(eye_rect_l[2:4]), color=(0,255,0), thickness=2)
            else:
                cv2.rectangle(frame, pt1=tuple(eye_rect_l[0:2]), pt2=tuple(eye_rect_l[2:4]), color=(0,0,255), thickness=2)
            
            #right
            if state_r == 1:
                cv2.rectangle(frame, pt1=tuple(eye_rect_r[0:2]), pt2=tuple(eye_rect_r[2:4]), color=(0,255,0), thickness=2)
            else:
                cv2.rectangle(frame, pt1=tuple(eye_rect_r[0:2]), pt2=tuple(eye_rect_r[2:4]), color=(0,0,255), thickness=2)
            
            # 알람 실행
            if state_l == 0 and state_r == 0:
                alarm_flags.append(True)
            elif state_l ==1 and state_r == 1:
                alarm_flags.append(False)
            else:
                alarm_flags.append(False)
                
        
        if True in alarm_flags and alarm.get_busy() == 0: # 0이면 음악이 재생되지 않고,아니면 재생중 ->  for 중복재생방지
            alarm.play(-1) # -1 : 음악이 반복 재생하게 함, flag라 생각하면 된다.
        elif True not in alarm_flags:
            alarm.stop()
            
            
        #frame 크기 조정
        #resize_frame = cv2.resize(text_frame ,None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR) # 키울 그림, 
        
        #그림 반전
        flip_frame = cv2.flip(frame,1)
        text_frame = cv2.putText(flip_frame, str(now), (10,30) ,cv2.FONT_HERSHEY_SIMPLEX,1, (0,255,255), 1, cv2.LINE_AA)
        resize_frame = cv2.resize(text_frame , dsize = (1920,1280), interpolation=cv2.INTER_LINEAR) # 키울 그림, dsize - 자신의 해상도 결정 

        cv2.imshow('cam', resize_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

pygame.mixer.quit()
stream.release()
cv2.destroyAllWindows()

In [ ]:
#end